In [45]:
!pip install hfst_dev
import hfst_dev
from hfst_dev import compose

In [46]:
!wget -O Latvian.lexc https://raw.githubusercontent.com/TomsLielkalns/DatZ1373-LV_MD1/main/Latvian.lexc

--2024-03-28 15:49:51--  https://raw.githubusercontent.com/TomsLielkalns/DatZ1373-LV_MD1/main/Latvian.lexc
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 810 [text/plain]
Saving to: ‘Latvian.lexc’

Latvian.lexc        100%[===================>]     810  --.-KB/s    in 0s      

2024-03-28 15:49:51 (39.9 MB/s) - ‘Latvian.lexc’ saved [810/810]



# Leksikona ģeneratora ielāde un izveide no datnes Latvian.lexc

In [47]:
generator = hfst_dev.compile_lexc_file('Latvian.lexc')

print(generator.lookup('med+N+Sg+Dat'))
print(generator.lookup('led+N+Sg+Nom'))
print(generator.lookup('tirg+N+Sg+Acc'))
print(generator.lookup('seg+N+Sg+Loc'))
print(generator.lookup('lamp+N+Sg+Gen'))
print(generator.lookup('som+N+Sg+Dat'))
print(generator.lookup('krūz+N+Sg+Nom'))
print(generator.lookup('saul+N+Sg+Dat'))
print(generator.lookup('zem+N+Sg+Nom'))
print(generator.lookup('lād+N+Sg+Dat'))

(('med^um', 0.0),)
(('led^us', 0.0),)
(('tirg^u', 0.0),)
(('seg^ā', 0.0),)
(('lamp^as', 0.0),)
(('som^ai', 0.0),)
(('krūz^e', 0.0),)
(('saul^ei', 0.0),)
(('zem^e', 0.0),)
(('lād^ei', 0.0),)


# Burtu aizvietošana
Burti tiek aizvietoti ar mīkstajiem, garajiem, vai citiem burtiem, kur nepieciešams.

In [48]:
ZReplacement = hfst_dev.regex('z -> ž ||  _ "^" M')
DtoZReplacement = hfst_dev.regex('d -> ž ||  _ "^" M')
LReplacement = hfst_dev.regex('l -> ļ ||  _ "^" M')
print(ZReplacement.lookup("krūz^Mu'"))
print(DtoZReplacement.lookup("lād^Mu"))
print(LReplacement.lookup("saul^Mu"))

(("krūž^Mu'", 0.0),)
(('lāž^Mu', 0.0),)
(('sauļ^Mu', 0.0),)


# "J" burta iespraušana vārdā "zeme"
Vārdam "zeme" nepieciešams iespraust burtu "J", lai neveidotos vārds "zemu"

In [49]:
InsertJ = hfst_dev.regex('[..] -> j || [b | f | m | p | v] _ "^" M')
print(InsertJ.lookup("zem^Mu'"))

(("zemj^Mu'", 0.0),)


# Palīgsimbolu dzēšana
Atbrīvojas no galotņu atzīmēšanas palīgsimboliem.

In [52]:
MCleanup = hfst_dev.regex('"^" M -> 0')
CleanupRoof = hfst_dev.regex('"^" -> 0')

cascadeDel = compose((generator, MCleanup, CleanupRoof))
print(cascadeDel.lookup("zem+N+Sg+Nom"))
print(cascadeDel.lookup("zem+N+Sg+Acc"))

(('zeme', 0.0),)
(('zemi', 0.0),)


# Izņēmuma gadījums vārdam "Sarakste"
Notiek līdzskaņu mija ģenetīva daudzskaitlī t->š, bet vārds "sarakste" ir izņēmums, un pareizi jābūt "sarakstu" nevis "sarakšu".

In [51]:
Exceptions = hfst_dev.regex('[[{sarakst} "+N" "+Pl" "+Gen" .x. {sarakstu}] ]')
print(Exceptions.lookup("sarakst+N+Pl+Gen"))

(('sarakstu', 0.0),)
